In [ ]:
# %pip install facebook_scraper pandas

In [ ]:
from facebook_scraper import *
import pandas as pd
import time
import numpy as np

In [ ]:
FANPAGE_LINK ="ClashofClans"
FOLDER_PATH = "../Data/Raw/"
COOKIE_PATH = "../config/cookies.txt"

PAGES_NUMBER = 30 # Number of pages to crawl

In [ ]:
results = [] 
start_url = None
def handle_pagination_url(url):
    global start_url
    start_url = url

In [ ]:
set_cookies(COOKIE_PATH)
while True:
    try:
        for post in get_posts(FANPAGE_LINK, pages=PAGES_NUMBER , start_url=start_url, request_url_callback=handle_pagination_url,
                              options={"comments": False, "reactions": True, 
                                       "reactors": False, "allow_extra_requests": True}):
            results.append(post)
        print("All done")
        break
    except exceptions.TemporarilyBanned:
        print("Temporarily banned, sleeping for 10m")
        if len(results) > 0:
            f = open("start_url.txt", "a")
            f.write(start_url + "\n")
            f.close()
        time.sleep(600)
        set_cookies(COOKIE_PATH)
    except exceptions.AccountDisabled:
        print("Temporarily banned, sleeping for 10m")
        if len(results) > 0:
            f = open("start_url.txt", "w")
            f.write(start_url + "\n")
            f.close()
        time.sleep(600)
        set_cookies(COOKIE_PATH)

In [ ]:
f = open("start_url.txt", "w")
f.write(start_url + "\n")
f.close()

In [ ]:
# Initialize dataframe to scrape Facebook post
post_df_full = pd.DataFrame(columns=results[0].keys(), index=range(len(results)), data=results)
# To df
path=FOLDER_PATH + 'ClashOfClans_Reactions'
post_df_full.to_csv(path + '.csv', index=False)
# print(path)
post_np_full = np.array(results)
np.save(path + ".npy", post_np_full)